# Phase 2: T⁶/ℤ₃ Orbifold - Premier Vrai CY₃

**Objectif**: Premier test réel de la conjecture λ₁ × H* = 6

**Runtime**: GPU recommandé pour N > 10000

---

## Contexte Mathématique

### L'orbifold T⁶/ℤ₃

T⁶/ℤ₃ est construit en quotientant T⁶ ≈ ℂ³/Λ par l'action:

$$g: (z_1, z_2, z_3) \mapsto (\omega z_1, \omega z_2, \omega z_3)$$

où $\omega = e^{2\pi i/3}$ est une racine cubique de l'unité.

### Propriétés

| Propriété | Valeur |
|-----------|--------|
| Holonomie | **SU(3)** |
| h¹¹ | 9 |
| h²¹ | 0 |
| χ | 18 |
| Points fixes | 27 (= 3³) |

### H* et Prédictions

| Définition | H* | λ₁ cible (si λ₁×H*=6) |
|------------|-----|------------------------|
| A: h¹¹ + h²¹ + 2 | 11 | 0.545 |
| B: h¹¹ + 2h²¹ + 4 | 13 | 0.462 |
| C: |h¹¹ - h²¹| + 2 | 11 | 0.545 |

---

In [ ]:
# Cell 1: Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import json
from datetime import datetime
import time
import matplotlib.pyplot as plt

# GPU optionnel
try:
    import cupy as cp
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("✓ CuPy available - GPU acceleration enabled")
except ImportError:
    GPU_AVAILABLE = False
    print("✗ CuPy not available - using CPU")

print(f"NumPy version: {np.__version__}")

In [ ]:
# Cell 2: Constants & Config

# T⁶/ℤ₃ Hodge numbers
H11, H21 = 9, 0
CHI = 18

# H* definitions
HSTAR_A = H11 + H21 + 2      # = 11
HSTAR_B = H11 + 2*H21 + 4    # = 13
HSTAR_C = abs(H11 - H21) + 2  # = 11

# Conjecture target
TARGET = 6  # λ₁ × H* = dim(SU(3)) - h = 8 - 2 = 6

# Target λ₁ for each H* definition
LAMBDA1_TARGET_A = TARGET / HSTAR_A  # = 6/11 ≈ 0.545
LAMBDA1_TARGET_B = TARGET / HSTAR_B  # = 6/13 ≈ 0.462
LAMBDA1_TARGET_C = TARGET / HSTAR_C  # = 6/11 ≈ 0.545

# Numerical parameters
N_VALUES = [1000, 2000, 5000, 10000]
K_VALUES = [15, 25, 40, 60]
SEEDS = [42, 123, 456]

# PASS/FAIL criteria
TOLERANCE = 0.15  # 15% tolerance for first real test

print(f"T⁶/ℤ₃ Orbifold Configuration:")
print(f"  Hodge numbers: h¹¹={H11}, h²¹={H21}")
print(f"  Euler: χ={CHI}")
print(f"  Holonomy: SU(3)")
print(f"\nH* definitions:")
print(f"  A: {HSTAR_A} → λ₁ target = {LAMBDA1_TARGET_A:.4f}")
print(f"  B: {HSTAR_B} → λ₁ target = {LAMBDA1_TARGET_B:.4f}")
print(f"  C: {HSTAR_C} → λ₁ target = {LAMBDA1_TARGET_C:.4f}")
print(f"\nConjecture target: λ₁ × H* = {TARGET}")

In [ ]:
# Cell 3: ℤ₃ Action & Orbifold Sampling

# ℤ₃ generator: ω = e^{2πi/3}
OMEGA = np.exp(2j * np.pi / 3)
OMEGA_ANGLE = 2 * np.pi / 3

def sample_T6_uniform(N: int, seed: int = 42) -> np.ndarray:
    """
    Sample N points uniformly on T⁶ = [0, 2π)⁶
    Returns angles in ℝ⁶.
    """
    rng = np.random.default_rng(seed)
    return rng.uniform(0, 2 * np.pi, size=(N, 6)).astype(np.float32)


def project_to_fundamental_domain(angles: np.ndarray) -> np.ndarray:
    """
    Project T⁶ points to the ℤ₃ fundamental domain.
    
    The ℤ₃ action on T⁶ = (S¹)⁶ (where we view ℂ³ as 3 pairs of real coords):
    g: (θ₁, θ₂, θ₃, θ₄, θ₅, θ₆) → (θ₁+2π/3, θ₂+2π/3, θ₃+2π/3, θ₄+2π/3, θ₅+2π/3, θ₆+2π/3)
    
    Fundamental domain: θ₁ ∈ [0, 2π/3)
    """
    angles_fund = angles.copy()
    
    # For each point, apply the ℤ₃ element that brings θ₁ into [0, 2π/3)
    theta1 = angles_fund[:, 0]
    
    # k = how many times we need to subtract 2π/3
    k = np.floor(theta1 / OMEGA_ANGLE).astype(int)
    
    # Apply g^{-k} to all coordinates
    for i in range(6):
        angles_fund[:, i] = np.mod(angles_fund[:, i] - k * OMEGA_ANGLE, 2 * np.pi)
    
    return angles_fund


def sample_T6_Z3_orbifold(N: int, seed: int = 42, method: str = "fundamental") -> np.ndarray:
    """
    Sample N points on T⁶/ℤ₃.
    
    method:
        "fundamental": Sample T⁶, project to fundamental domain
        "oversample": Sample 3N on T⁶, keep N in fundamental domain
    """
    if method == "fundamental":
        # Sample on full T⁶
        angles = sample_T6_uniform(N, seed)
        # Project to fundamental domain
        return project_to_fundamental_domain(angles)
    
    elif method == "oversample":
        # Oversample to ensure good coverage
        angles = sample_T6_uniform(N * 3, seed)
        # Keep only those in fundamental domain
        mask = angles[:, 0] < OMEGA_ANGLE
        angles_fund = angles[mask]
        # Return exactly N points
        return angles_fund[:N]
    
    else:
        raise ValueError(f"Unknown method: {method}")


print("✓ T⁶/ℤ₃ sampling functions defined")
print(f"  ℤ₃ generator ω = e^{{2πi/3}}, angle = {OMEGA_ANGLE:.4f} rad")

In [ ]:
# Cell 4: Orbifold Distance

def geodesic_distance_T6(angles1: np.ndarray, angles2: np.ndarray) -> np.ndarray:
    """
    Geodesic distance between two sets of points on T⁶.
    
    Returns: (N1, N2) distance matrix
    """
    # Broadcast: (N1, 1, 6) - (1, N2, 6) = (N1, N2, 6)
    diff = np.abs(angles1[:, None, :] - angles2[None, :, :])
    
    # Toric distance on each S¹
    diff_toric = np.minimum(diff, 2 * np.pi - diff)
    
    # Euclidean in product
    return np.sqrt(np.sum(diff_toric**2, axis=2))


def geodesic_distance_T6_Z3(angles: np.ndarray) -> np.ndarray:
    """
    Distance matrix on T⁶/ℤ₃ orbifold.
    
    For each pair of points, the distance is the minimum over all ℤ₃ images:
    d(p, q) = min_{g ∈ ℤ₃} d_T⁶(p, g·q)
    """
    N = angles.shape[0]
    
    # Initialize with large values
    D_min = np.full((N, N), np.inf, dtype=np.float32)
    
    # Try all 3 elements of ℤ₃
    for k in range(3):
        # Apply g^k: shift all angles by k * 2π/3
        angles_shifted = np.mod(angles + k * OMEGA_ANGLE, 2 * np.pi)
        
        # Compute T⁶ distance
        D_k = geodesic_distance_T6(angles, angles_shifted)
        
        # Keep minimum
        D_min = np.minimum(D_min, D_k)
    
    return D_min


print("✓ Orbifold distance functions defined")

In [ ]:
# Cell 5: Graph Laplacian (reused from Phase 1)

def build_graph_laplacian(D: np.ndarray, k: int = 25,
                          laplacian_type: str = "symmetric") -> sp.csr_matrix:
    """
    Build graph Laplacian from distance matrix.
    """
    N = D.shape[0]
    k = min(k, N - 1)
    
    # Sigma = median of k-NN distances
    knn_dists = np.partition(D, k, axis=1)[:, 1:k+1]
    sigma = max(np.median(knn_dists), 1e-10)
    
    # Gaussian weights
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # Keep only k nearest neighbors
    for i in range(N):
        idx_keep = np.argpartition(-W[i], k)[:k]
        mask = np.ones(N, dtype=bool)
        mask[idx_keep] = False
        W[i, mask] = 0
    
    # Symmetrize
    W = (W + W.T) / 2
    
    # Degrees
    d = np.maximum(W.sum(axis=1), 1e-10)
    
    # Build Laplacian
    if laplacian_type == "symmetric":
        d_inv_sqrt = 1.0 / np.sqrt(d)
        L = np.eye(N) - np.outer(d_inv_sqrt, d_inv_sqrt) * W
    elif laplacian_type == "random_walk":
        d_inv = 1.0 / d
        L = np.eye(N) - np.diag(d_inv) @ W
    else:
        L = np.diag(d) - W
    
    return sp.csr_matrix(L)


def compute_lambda1(L: sp.csr_matrix, n_eigenvalues: int = 5) -> float:
    """
    Compute λ₁ (first non-zero eigenvalue).
    """
    try:
        eigs, _ = eigsh(L, k=n_eigenvalues, which='SM', tol=1e-8)
        eigs = np.sort(np.real(eigs))
        
        for ev in eigs:
            if ev > 1e-8:
                return float(ev)
        
        return float(eigs[1]) if len(eigs) > 1 else 0.0
        
    except Exception as e:
        print(f"Eigensolve error: {e}")
        return np.nan


print("✓ Graph Laplacian functions ready")

In [ ]:
# Cell 6: Quick Test
print("=" * 60)
print("  QUICK TEST: N=2000")
print("=" * 60)

t0 = time.time()

# Sample on orbifold
angles = sample_T6_Z3_orbifold(2000, seed=42)
print(f"Sampled {len(angles)} points on T⁶/ℤ₃")
print(f"  θ₁ range: [{angles[:, 0].min():.4f}, {angles[:, 0].max():.4f}] (should be in [0, 2π/3))")

# Orbifold distance
D = geodesic_distance_T6_Z3(angles)
print(f"Distance matrix: {D.shape}, range [{D.min():.3f}, {D.max():.3f}]")

# Laplacian & eigenvalue
L = build_graph_laplacian(D, k=25)
lambda1 = compute_lambda1(L)

# Test all H* definitions
print(f"\nResults:")
print(f"  λ₁ (measured): {lambda1:.4f}")
print(f"  Time: {time.time() - t0:.2f}s")

print(f"\nConjecture test (λ₁ × H* = 6):")
for name, Hstar, target in [("A", HSTAR_A, LAMBDA1_TARGET_A), 
                             ("B", HSTAR_B, LAMBDA1_TARGET_B),
                             ("C", HSTAR_C, LAMBDA1_TARGET_C)]:
    product = lambda1 * Hstar
    deviation = abs(product - TARGET) / TARGET * 100
    status = "✓" if deviation < TOLERANCE * 100 else "?"
    print(f"  H*_{name}={Hstar}: λ₁×H* = {product:.3f} (dev={deviation:.1f}%) [{status}]")

In [ ]:
# Cell 7: Full Validation Grid
print("=" * 60)
print("  FULL VALIDATION: T⁶/ℤ₃ ORBIFOLD")
print("=" * 60)
print(f"Grid: N={N_VALUES}, k={K_VALUES}")
print(f"Seeds: {SEEDS}")
print(f"Total tests: {len(N_VALUES) * len(K_VALUES) * len(SEEDS)}")
print()

results = []
t_start = time.time()

for N in N_VALUES:
    print(f"\nN = {N}...")
    
    for k in K_VALUES:
        for seed in SEEDS:
            t0 = time.time()
            
            # Sample & compute
            angles = sample_T6_Z3_orbifold(N, seed=seed)
            D = geodesic_distance_T6_Z3(angles)
            L = build_graph_laplacian(D, k=k)
            lambda1 = compute_lambda1(L)
            
            elapsed = time.time() - t0
            
            # Test all H* definitions
            for Hstar_name, Hstar in [("A", HSTAR_A), ("B", HSTAR_B), ("C", HSTAR_C)]:
                product = lambda1 * Hstar
                deviation = abs(product - TARGET) / TARGET * 100
                
                result = {
                    "N": N,
                    "k": k,
                    "seed": seed,
                    "Hstar_def": Hstar_name,
                    "Hstar": Hstar,
                    "lambda1": float(lambda1),
                    "product": float(product),
                    "deviation_from_6_pct": float(deviation),
                    "passed": deviation < TOLERANCE * 100,
                    "time_s": elapsed,
                }
                results.append(result)
            
            # Print one line per (N, k, seed)
            products = [lambda1 * H for H in [HSTAR_A, HSTAR_B, HSTAR_C]]
            print(f"  k={k}, seed={seed}: λ₁={lambda1:.4f} → products={[f'{p:.2f}' for p in products]} [{elapsed:.1f}s]")

t_total = time.time() - t_start
print(f"\nTotal time: {t_total/60:.1f} minutes")

In [ ]:
# Cell 8: Analysis by H* Definition
print("=" * 60)
print("  ANALYSIS: WHICH H* DEFINITION WORKS?")
print("=" * 60)

# Group by H* definition
analysis = {}
for Hstar_name in ["A", "B", "C"]:
    subset = [r for r in results if r["Hstar_def"] == Hstar_name]
    products = [r["product"] for r in subset]
    deviations = [r["deviation_from_6_pct"] for r in subset]
    
    # High-N only (more reliable)
    high_N_subset = [r for r in subset if r["N"] >= 5000]
    high_N_products = [r["product"] for r in high_N_subset] if high_N_subset else products
    
    analysis[Hstar_name] = {
        "Hstar": subset[0]["Hstar"],
        "mean_product": np.mean(products),
        "std_product": np.std(products),
        "mean_deviation": np.mean(deviations),
        "high_N_mean_product": np.mean(high_N_products),
        "high_N_std_product": np.std(high_N_products) if len(high_N_products) > 1 else 0,
        "n_passed": sum(1 for r in subset if r["passed"]),
        "n_total": len(subset),
    }

print(f"\nTarget: λ₁ × H* = {TARGET}")
print(f"Tolerance: ±{TOLERANCE*100:.0f}%")
print()

print("| H* Def | H* | Mean Product | Std | High-N Mean | Pass Rate |")
print("|--------|-----|--------------|-----|-------------|-----------|")
for name, data in analysis.items():
    pass_rate = data["n_passed"] / data["n_total"] * 100
    print(f"| {name}      | {data['Hstar']:3d} | {data['mean_product']:12.3f} | {data['std_product']:.3f} | {data['high_N_mean_product']:11.3f} | {pass_rate:8.1f}% |")

# Identify winner
winner = min(analysis.keys(), key=lambda k: abs(analysis[k]["high_N_mean_product"] - TARGET))
print(f"\n→ Best match: H*_{winner} (closest to {TARGET})")

In [ ]:
# Cell 9: Visualization
print("Generating plots...")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: λ₁ vs N for each k
ax1 = axes[0]
for k in K_VALUES:
    subset = [r for r in results if r["k"] == k and r["Hstar_def"] == "A"]
    # Average over seeds
    by_N = {}
    for r in subset:
        by_N.setdefault(r["N"], []).append(r["lambda1"])
    Ns = sorted(by_N.keys())
    means = [np.mean(by_N[N]) for N in Ns]
    stds = [np.std(by_N[N]) for N in Ns]
    ax1.errorbar(Ns, means, yerr=stds, marker='o', label=f'k={k}')

ax1.axhline(y=LAMBDA1_TARGET_A, color='r', linestyle='--', label=f'λ₁ target (A) = {LAMBDA1_TARGET_A:.3f}')
ax1.axhline(y=LAMBDA1_TARGET_B, color='g', linestyle='--', label=f'λ₁ target (B) = {LAMBDA1_TARGET_B:.3f}')
ax1.set_xlabel('N')
ax1.set_ylabel('λ₁')
ax1.set_title('T⁶/ℤ₃: λ₁ vs N')
ax1.set_xscale('log')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)

# Plot 2: λ₁×H* products for each definition
ax2 = axes[1]
colors = {'A': 'blue', 'B': 'green', 'C': 'orange'}
for Hstar_name in ["A", "B", "C"]:
    subset = [r for r in results if r["Hstar_def"] == Hstar_name]
    by_N = {}
    for r in subset:
        by_N.setdefault(r["N"], []).append(r["product"])
    Ns = sorted(by_N.keys())
    means = [np.mean(by_N[N]) for N in Ns]
    ax2.plot(Ns, means, 'o-', color=colors[Hstar_name], label=f'H*_{Hstar_name}')

ax2.axhline(y=TARGET, color='r', linestyle='--', linewidth=2, label=f'Target = {TARGET}')
ax2.axhspan(TARGET * (1 - TOLERANCE), TARGET * (1 + TOLERANCE), alpha=0.2, color='green')
ax2.set_xlabel('N')
ax2.set_ylabel('λ₁ × H*')
ax2.set_title('T⁶/ℤ₃: Conjecture Test')
ax2.set_xscale('log')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Deviation from target
ax3 = axes[2]
for Hstar_name in ["A", "B", "C"]:
    subset = [r for r in results if r["Hstar_def"] == Hstar_name]
    by_N = {}
    for r in subset:
        by_N.setdefault(r["N"], []).append(r["deviation_from_6_pct"])
    Ns = sorted(by_N.keys())
    means = [np.mean(by_N[N]) for N in Ns]
    ax3.plot(Ns, means, 'o-', color=colors[Hstar_name], label=f'H*_{Hstar_name}')

ax3.axhline(y=TOLERANCE * 100, color='r', linestyle='--', label=f'PASS threshold ({TOLERANCE*100:.0f}%)')
ax3.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax3.set_xlabel('N')
ax3.set_ylabel('Deviation from 6 (%)')
ax3.set_title('T⁶/ℤ₃: Convergence to Target')
ax3.set_xscale('log')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('T6_Z3_validation.png', dpi=150)
plt.show()

print("✓ Saved: T6_Z3_validation.png")

In [ ]:
# Cell 10: Summary & Decision
print("=" * 60)
print("  PHASE 2 SUMMARY: T⁶/ℤ₃ ORBIFOLD")
print("=" * 60)

# Best definition
best_def = winner
best_data = analysis[best_def]

print(f"\nManifold: T⁶/ℤ₃")
print(f"  Holonomy: SU(3)")
print(f"  h¹¹={H11}, h²¹={H21}, χ={CHI}")

print(f"\nConjecture: λ₁ × H* = {TARGET} = dim(SU(3)) - h")

print(f"\nBest H* definition: {best_def}")
print(f"  H* = {best_data['Hstar']}")
print(f"  Mean product: {best_data['mean_product']:.3f}")
print(f"  High-N mean: {best_data['high_N_mean_product']:.3f}")
print(f"  Deviation from {TARGET}: {abs(best_data['high_N_mean_product'] - TARGET) / TARGET * 100:.1f}%")

# PASS/FAIL
best_deviation = abs(best_data['high_N_mean_product'] - TARGET) / TARGET * 100
phase2_passed = best_deviation < TOLERANCE * 100

print("\n" + "-" * 40)
if phase2_passed:
    print(f"  PHASE 2 STATUS: [PASS]")
    print(f"  T⁶/ℤ₃ satisfies λ₁ × H*_{best_def} ≈ {TARGET}")
    print(f"  Prêt pour Phase 3: CICY (Quintic, Bicubic)")
else:
    print(f"  PHASE 2 STATUS: [INCONCLUSIVE]")
    print(f"  Deviation {best_deviation:.1f}% > tolerance {TOLERANCE*100:.0f}%")
    print(f"  Options: augmenter N, affiner méthode, ou conjecture incorrecte")
print("-" * 40)

In [ ]:
# Cell 11: Save Results
output = {
    "metadata": {
        "phase": "Phase 2: T⁶/ℤ₃ Orbifold",
        "timestamp": datetime.now().isoformat(),
        "manifold": "T6_Z3",
        "holonomy": "SU(3)",
        "h11": H11,
        "h21": H21,
        "chi": CHI,
        "conjecture_target": TARGET,
        "Hstar_definitions": {
            "A": {"formula": "h11 + h21 + 2", "value": HSTAR_A},
            "B": {"formula": "h11 + 2*h21 + 4", "value": HSTAR_B},
            "C": {"formula": "|h11 - h21| + 2", "value": HSTAR_C},
        },
        "N_values": N_VALUES,
        "k_values": K_VALUES,
        "seeds": SEEDS,
        "tolerance": TOLERANCE,
        "total_time_minutes": t_total / 60,
    },
    "results": results,
    "analysis": analysis,
    "summary": {
        "best_Hstar_definition": best_def,
        "best_Hstar_value": best_data["Hstar"],
        "best_mean_product": best_data["high_N_mean_product"],
        "best_deviation_pct": best_deviation,
        "phase2_passed": phase2_passed,
    },
    "next_phase": "Phase 3: CICY (Quintic, Bicubic)" if phase2_passed else "Review methodology",
}

filename = "T6_Z3_validation_results.json"
with open(filename, "w") as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Saved: {filename}")
print("\nDownload this file and the plot, then share with Claude!")

In [ ]:
# Cell 12: Comparison with G₂ Results
print("\n" + "=" * 60)
print("  CROSS-VALIDATION: CY₃ vs G₂")
print("=" * 60)

# G₂ reference (from N=50000 validation)
G2_RESULT = {
    "manifold": "K₇ (TCS)",
    "holonomy": "G₂",
    "dim_Hol": 14,
    "h_spinors": 1,
    "Hstar": 99,
    "target": 13,
    "measured": 13.0,  # Exact at N=50000, k=165
    "deviation_pct": 0.0,
}

# CY₃ result
CY3_RESULT = {
    "manifold": "T⁶/ℤ₃",
    "holonomy": "SU(3)",
    "dim_Hol": 8,
    "h_spinors": 2,
    "Hstar": best_data["Hstar"],
    "target": 6,
    "measured": best_data["high_N_mean_product"],
    "deviation_pct": best_deviation,
}

print(f"\n| Manifold | Holonomy | dim(Hol) | h | H* | Target | Measured | Dev |")
print(f"|----------|----------|----------|---|-----|--------|----------|-----|")
print(f"| {G2_RESULT['manifold']} | {G2_RESULT['holonomy']} | {G2_RESULT['dim_Hol']} | {G2_RESULT['h_spinors']} | {G2_RESULT['Hstar']} | {G2_RESULT['target']} | {G2_RESULT['measured']:.2f} | {G2_RESULT['deviation_pct']:.1f}% |")
print(f"| {CY3_RESULT['manifold']} | {CY3_RESULT['holonomy']} | {CY3_RESULT['dim_Hol']} | {CY3_RESULT['h_spinors']} | {CY3_RESULT['Hstar']} | {CY3_RESULT['target']} | {CY3_RESULT['measured']:.2f} | {CY3_RESULT['deviation_pct']:.1f}% |")

print(f"\nUniversal Law Test: λ₁ × H* = dim(Hol) - h")
print(f"  G₂:   14 - 1 = 13 → {G2_RESULT['measured']:.2f} {'[✓]' if G2_RESULT['deviation_pct'] < 10 else '[?]'}")
print(f"  SU(3): 8 - 2 = 6  → {CY3_RESULT['measured']:.2f} {'[✓]' if CY3_RESULT['deviation_pct'] < 15 else '[?]'}")